# Elexon and NESO Data Downloader

This notebook will help you download data from Elexon and NESO APIs that hasn't been downloaded yet. Instead of using ZeroMQ messaging (which was the focus of the previous version), we'll use a direct approach to:

1. Connect to Elexon and NESO APIs
2. Identify missing data
3. Download the data efficiently
4. Store it locally and/or in Google Cloud Storage

Let's get started with the implementation.

In [22]:
# Import required libraries for Elexon and NESO data download
import os
import json
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional
import logging

# Optional: Import Google Cloud libraries if you want to upload to GCS
try:
    from google.cloud import storage
    gcs_available = True
except ImportError:
    gcs_available = False
    print("Google Cloud Storage libraries not available. Data will be stored locally only.")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('elexon_neso_downloader')

In [23]:
# Configuration for Elexon and NESO APIs
# Replace with your actual API keys and customize settings as needed

# Elexon API Configuration
ELEXON_API_BASE = 'https://data.elexon.co.uk/bmrs/api/v1/'
ELEXON_API_KEY = os.getenv('ELEXON_API_KEY', '')  # Set your API key in environment variable or directly here

# NESO API Configuration
NESO_API_BASE = 'https://data.nationalgrideso.com/api/v1/'
NESO_API_KEY = os.getenv('NESO_API_KEY', '')  # Set your API key if required

# Google Cloud Configuration (if available)
GCS_BUCKET_NAME = os.getenv('GCS_BUCKET_NAME', 'jibber-jabber-knowledge-bmrs-data')
GCS_PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT', 'jibber-jabber-knowledge')

# Local storage configuration
LOCAL_DATA_DIR = Path('./downloaded_data')
LOCAL_DATA_DIR.mkdir(exist_ok=True)

# Print current configuration (without exposing API keys)
print(f"Elexon API Base: {ELEXON_API_BASE}")
print(f"NESO API Base: {NESO_API_BASE}")
print(f"GCS Available: {gcs_available}")
print(f"Local Data Directory: {LOCAL_DATA_DIR.absolute()}")
print(f"Elexon API Key Configured: {'Yes' if ELEXON_API_KEY else 'No'}")
print(f"NESO API Key Configured: {'Yes' if NESO_API_KEY else 'No'}")

Elexon API Base: https://data.elexon.co.uk/bmrs/api/v1/
NESO API Base: https://data.nationalgrideso.com/api/v1/
GCS Available: True
Local Data Directory: /Users/georgemajor/Jibber Jabber ChatGPT/8_august_jibber_jabber/downloaded_data
Elexon API Key Configured: No
NESO API Key Configured: No


In [24]:
# Utility functions for API requests with error handling and retries

def create_session_with_retries(retries=3, backoff_factor=0.5):
    """Create a requests session with retry logic"""
    session = requests.Session()
    retry = requests.adapters.Retry(
        total=retries,
        backoff_factor=backoff_factor,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"]
    )
    adapter = requests.adapters.HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def make_elexon_api_request(endpoint, params=None):
    """Make a request to the Elexon API with error handling and retries"""
    if params is None:
        params = {}
    
    # Add API key to parameters if provided
    if ELEXON_API_KEY:
        params['APIKey'] = ELEXON_API_KEY
    
    url = f"{ELEXON_API_BASE.rstrip('/')}/{endpoint.lstrip('/')}"
    session = create_session_with_retries()
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error making request to Elexon API {url}: {e}")
        return None
    
def make_neso_api_request(endpoint, params=None):
    """Make a request to the NESO API with error handling and retries"""
    if params is None:
        params = {}
    
    # Add API key to parameters if provided
    if NESO_API_KEY:
        params['APIKey'] = NESO_API_KEY
    
    url = f"{NESO_API_BASE.rstrip('/')}/{endpoint.lstrip('/')}"
    session = create_session_with_retries()
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error making request to NESO API {url}: {e}")
        return None
        
def save_json_data(data, dataset_name, date_str, is_elexon=True):
    """Save JSON data to a file with proper naming convention"""
    source = "elexon" if is_elexon else "neso"
    filename = f"{source}_{dataset_name}_{date_str}.json"
    filepath = LOCAL_DATA_DIR / filename
    
    with open(filepath, 'w') as f:
        json.dump(data, f)
        
    logger.info(f"Saved data to {filepath}")
    return filepath

def save_csv_data(data, dataset_name, date_str, is_elexon=True):
    """Save data as CSV file"""
    source = "elexon" if is_elexon else "neso"
    filename = f"{source}_{dataset_name}_{date_str}.csv"
    filepath = LOCAL_DATA_DIR / filename
    
    # Convert to DataFrame if it's not already
    if not isinstance(data, pd.DataFrame):
        df = pd.DataFrame(data)
    else:
        df = data
        
    df.to_csv(filepath, index=False)
    logger.info(f"Saved CSV data to {filepath}")
    return filepath

def upload_to_gcs(local_filepath, bucket_name=GCS_BUCKET_NAME):
    """Upload a file to Google Cloud Storage if available"""
    if not gcs_available:
        logger.warning("GCS libraries not available. Skipping upload.")
        return False
    
    try:
        storage_client = storage.Client(project=GCS_PROJECT_ID)
        bucket = storage_client.bucket(bucket_name)
        blob_name = local_filepath.name
        blob = bucket.blob(blob_name)
        
        blob.upload_from_filename(local_filepath)
        logger.info(f"Uploaded {local_filepath} to gs://{bucket_name}/{blob_name}")
        return True
    except Exception as e:
        logger.error(f"Error uploading to GCS: {e}")
        return False

In [25]:
# Elexon API data download functions

def get_elexon_datasets():
    """Get list of available datasets from Elexon API"""
    data = make_elexon_api_request('datasets')
    if data and 'data' in data:
        return data['data']
    return []

def get_elexon_dataset_details(dataset_id):
    """Get details about a specific Elexon dataset"""
    data = make_elexon_api_request(f'datasets/{dataset_id}')
    if data:
        return data
    return None

def check_data_exists(dataset_id, date_str):
    """Check if data already exists locally or in GCS for a given date"""
    local_json_file = LOCAL_DATA_DIR / f"elexon_{dataset_id}_{date_str}.json"
    local_csv_file = LOCAL_DATA_DIR / f"elexon_{dataset_id}_{date_str}.csv"
    
    if local_json_file.exists() or local_csv_file.exists():
        return True
    
    # Could also check GCS if needed
    return False

def download_elexon_data(dataset_id, start_date, end_date=None, format='json'):
    """
    Download data for a specific Elexon dataset between start_date and end_date
    
    Args:
        dataset_id: The Elexon dataset ID (e.g., 'FUELINST')
        start_date: Start date in 'YYYY-MM-DD' format
        end_date: End date in 'YYYY-MM-DD' format (defaults to start_date if None)
        format: Output format ('json' or 'csv')
    
    Returns:
        List of downloaded file paths
    """
    if end_date is None:
        end_date = start_date
        
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    downloaded_files = []
    current_dt = start_dt
    
    # Iterate through each day in the date range
    while current_dt <= end_dt:
        date_str = current_dt.strftime("%Y-%m-%d")
        
        # Skip if data already exists
        if check_data_exists(dataset_id, date_str):
            logger.info(f"Data for {dataset_id} on {date_str} already exists. Skipping.")
            current_dt += timedelta(days=1)
            continue
        
        # Set up parameters for the API request
        params = {
            'from': f"{date_str}T00:00:00Z",
            'to': f"{date_str}T23:59:59Z",
            'format': format
        }
        
        logger.info(f"Downloading {dataset_id} data for {date_str}")
        
        # Make the API request
        endpoint = f"datasets/{dataset_id}"
        data = make_elexon_api_request(endpoint, params)
        
        if data:
            # Save the data
            if format.lower() == 'json':
                filepath = save_json_data(data, dataset_id, date_str)
            else:
                # For CSV, we might need to extract the data differently
                if 'data' in data:
                    df = pd.DataFrame(data['data'])
                    filepath = save_csv_data(df, dataset_id, date_str)
                else:
                    logger.warning(f"Unexpected data format for {dataset_id}. Saving as JSON instead.")
                    filepath = save_json_data(data, dataset_id, date_str)
            
            # Upload to GCS if available
            if gcs_available:
                upload_to_gcs(filepath)
                
            downloaded_files.append(filepath)
            
        # Add a small delay to avoid API rate limits
        time.sleep(1)
        current_dt += timedelta(days=1)
    
    return downloaded_files

# List of commonly used Elexon datasets
COMMON_ELEXON_DATASETS = [
    'FUELINST',  # Generation by fuel type
    'DEMMF',     # Demand forecast
    'B1610',     # Actual generation output per unit
    'TEMP',      # Temperature data
    'WINDFOR',   # Wind forecast
    'BOD',       # Bid-offer data
    'MID',       # Market index data
    'SYSWARN'    # System warnings
]

def download_all_missing_elexon_data(datasets=COMMON_ELEXON_DATASETS, days_back=30):
    """
    Download all missing data for specified Elexon datasets for the last N days
    
    Args:
        datasets: List of dataset IDs to download
        days_back: Number of days back from today to check
    """
    end_date = datetime.now().strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")
    
    results = {}
    
    for dataset_id in datasets:
        logger.info(f"Processing dataset {dataset_id}")
        try:
            files = download_elexon_data(dataset_id, start_date, end_date)
            results[dataset_id] = len(files)
        except Exception as e:
            logger.error(f"Error downloading {dataset_id}: {e}")
            results[dataset_id] = f"Error: {str(e)}"
    
    return results

In [26]:
# NESO API data download functions

def get_neso_datasets():
    """Get list of available datasets from NESO API"""
    data = make_neso_api_request('datasets')
    if data and 'data' in data:
        return data['data']
    return []

def check_neso_data_exists(dataset_id, date_str):
    """Check if NESO data already exists locally or in GCS for a given date"""
    local_json_file = LOCAL_DATA_DIR / f"neso_{dataset_id}_{date_str}.json"
    local_csv_file = LOCAL_DATA_DIR / f"neso_{dataset_id}_{date_str}.csv"
    
    if local_json_file.exists() or local_csv_file.exists():
        return True
    
    # Could also check GCS if needed
    return False

def download_neso_data(dataset_id, start_date, end_date=None, format='json'):
    """
    Download data for a specific NESO dataset between start_date and end_date
    
    Args:
        dataset_id: The NESO dataset ID
        start_date: Start date in 'YYYY-MM-DD' format
        end_date: End date in 'YYYY-MM-DD' format (defaults to start_date if None)
        format: Output format ('json' or 'csv')
    
    Returns:
        List of downloaded file paths
    """
    if end_date is None:
        end_date = start_date
        
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    downloaded_files = []
    current_dt = start_dt
    
    # Iterate through each day in the date range
    while current_dt <= end_dt:
        date_str = current_dt.strftime("%Y-%m-%d")
        
        # Skip if data already exists
        if check_neso_data_exists(dataset_id, date_str):
            logger.info(f"NESO data for {dataset_id} on {date_str} already exists. Skipping.")
            current_dt += timedelta(days=1)
            continue
        
        # Set up parameters for the API request
        params = {
            'from': f"{date_str}T00:00:00Z",
            'to': f"{date_str}T23:59:59Z",
            'format': format
        }
        
        logger.info(f"Downloading NESO {dataset_id} data for {date_str}")
        
        # Make the API request
        endpoint = f"datasets/{dataset_id}"
        data = make_neso_api_request(endpoint, params)
        
        if data:
            # Save the data
            if format.lower() == 'json':
                filepath = save_json_data(data, dataset_id, date_str, is_elexon=False)
            else:
                # For CSV, we might need to extract the data differently
                if 'data' in data:
                    df = pd.DataFrame(data['data'])
                    filepath = save_csv_data(df, dataset_id, date_str, is_elexon=False)
                else:
                    logger.warning(f"Unexpected NESO data format for {dataset_id}. Saving as JSON instead.")
                    filepath = save_json_data(data, dataset_id, date_str, is_elexon=False)
            
            # Upload to GCS if available
            if gcs_available:
                upload_to_gcs(filepath)
                
            downloaded_files.append(filepath)
            
        # Add a small delay to avoid API rate limits
        time.sleep(1)
        current_dt += timedelta(days=1)
    
    return downloaded_files

# List of commonly used NESO datasets
COMMON_NESO_DATASETS = [
    'demand-data',
    'generation-mix',
    'carbon-intensity',
    'frequency-data',
    'embedded-wind-and-solar-forecasts',
    'transmission-system-warnings'
]

def download_all_missing_neso_data(datasets=COMMON_NESO_DATASETS, days_back=30):
    """
    Download all missing data for specified NESO datasets for the last N days
    
    Args:
        datasets: List of dataset IDs to download
        days_back: Number of days back from today to check
    """
    end_date = datetime.now().strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")
    
    results = {}
    
    for dataset_id in datasets:
        logger.info(f"Processing NESO dataset {dataset_id}")
        try:
            files = download_neso_data(dataset_id, start_date, end_date)
            results[dataset_id] = len(files)
        except Exception as e:
            logger.error(f"Error downloading NESO {dataset_id}: {e}")
            results[dataset_id] = f"Error: {str(e)}"
    
    return results

## Download Missing Data from Elexon and NESO

Now let's use the functions we've created to:

1. List available datasets
2. Check for missing data
3. Download the missing data

You can adjust the date ranges and datasets as needed.

In [27]:
# 1. First, let's list available Elexon datasets
# Uncomment to run (note: might be rate-limited without an API key)
# elexon_datasets = get_elexon_datasets()
# print(f"Found {len(elexon_datasets)} Elexon datasets")
# for i, dataset in enumerate(elexon_datasets[:10]):  # Show first 10
#     print(f"{i+1}. {dataset.get('datasetId', 'Unknown')}: {dataset.get('name', 'No name')}")

# 2. Download missing data for common Elexon datasets for the last 7 days
# Customize the list and date range as needed
selected_elexon_datasets = ['FUELINST', 'DEMMF', 'TEMP']  # Subset for testing
days_to_check = 7  # Last 7 days

print(f"Will check for missing Elexon data in these datasets: {selected_elexon_datasets}")
print(f"Date range: Last {days_to_check} days")

# Uncomment to execute the download (ensure you have set ELEXON_API_KEY)
# elexon_results = download_all_missing_elexon_data(
#     datasets=selected_elexon_datasets,
#     days_back=days_to_check
# )
# 
# print("\nElexon download results:")
# for dataset, count in elexon_results.items():
#     print(f"{dataset}: {'Error' if isinstance(count, str) else f'{count} files downloaded'}")

Will check for missing Elexon data in these datasets: ['FUELINST', 'DEMMF', 'TEMP']
Date range: Last 7 days


In [28]:
# 3. Download missing data for NESO datasets for the last 7 days
# Customize the list and date range as needed
selected_neso_datasets = ['actual_generation_per_unit', 'forecast_demand_published']  # Subset for testing
days_to_check = 7  # Last 7 days

print(f"Will check for missing NESO data in these datasets: {selected_neso_datasets}")
print(f"Date range: Last {days_to_check} days")

# Uncomment to execute the download (ensure you have set NESO_API_KEY)
# neso_results = download_all_missing_neso_data(
#     datasets=selected_neso_datasets,
#     days_back=days_to_check
# )
# 
# print("\nNESO download results:")
# for dataset, count in neso_results.items():
#     print(f"{dataset}: {'Error' if isinstance(count, str) else f'{count} files downloaded'}")

Will check for missing NESO data in these datasets: ['actual_generation_per_unit', 'forecast_demand_published']
Date range: Last 7 days


In [29]:
# 4. Comprehensive download function to get all missing data from both systems
def download_all_missing_data(elexon_datasets=None, neso_datasets=None, days_back=7, 
                             use_gcs=False, bucket_name=None):
    """
    Download all missing data from both Elexon and NESO APIs
    
    Args:
        elexon_datasets: List of Elexon datasets to check
        neso_datasets: List of NESO datasets to check
        days_back: Number of days to look back for missing data
        use_gcs: Whether to also upload to Google Cloud Storage
        bucket_name: GCS bucket name if use_gcs is True
        
    Returns:
        dict: Summary of downloaded files for each dataset
    """
    results = {}
    
    # If not specified, use common datasets for each system
    if elexon_datasets is None:
        elexon_datasets = ['DEMMF', 'FUELINST', 'TEMP', 'B1610', 'SYSWARN']
    
    if neso_datasets is None:
        neso_datasets = ['actual_generation_per_unit', 'forecast_demand_published', 
                         'system_warnings', 'system_alerts']
    
    # Download Elexon data
    print(f"Downloading missing Elexon data for {len(elexon_datasets)} datasets...")
    elexon_results = download_all_missing_elexon_data(
        datasets=elexon_datasets,
        days_back=days_back,
        use_gcs=use_gcs,
        bucket_name=bucket_name
    )
    results['elexon'] = elexon_results
    
    # Download NESO data
    print(f"\nDownloading missing NESO data for {len(neso_datasets)} datasets...")
    neso_results = download_all_missing_neso_data(
        datasets=neso_datasets,
        days_back=days_back,
        use_gcs=use_gcs,
        bucket_name=bucket_name
    )
    results['neso'] = neso_results
    
    # Print summary
    total_files = 0
    total_errors = 0
    
    print("\n===== DOWNLOAD SUMMARY =====")
    print("\nELEXON RESULTS:")
    for dataset, count in elexon_results.items():
        if isinstance(count, str):
            print(f"  {dataset}: ERROR - {count}")
            total_errors += 1
        else:
            print(f"  {dataset}: {count} files downloaded")
            total_files += count
    
    print("\nNESO RESULTS:")
    for dataset, count in neso_results.items():
        if isinstance(count, str):
            print(f"  {dataset}: ERROR - {count}")
            total_errors += 1
        else:
            print(f"  {dataset}: {count} files downloaded")
            total_files += count
    
    print(f"\nTOTAL: {total_files} files downloaded, {total_errors} datasets with errors")
    return results

# Uncomment to run a full download of missing data from both systems
# all_results = download_all_missing_data(
#     days_back=7,  # Last 7 days
#     use_gcs=False  # Set to True if you want to upload to GCS
# )

In [30]:
# 5. Analyze and validate downloaded data
import pandas as pd
import os
import json
from datetime import datetime, timedelta

def analyze_downloaded_data(base_dir='./data', days_back=7):
    """
    Analyze the downloaded data files to verify completeness and quality
    
    Args:
        base_dir: Base directory where data is stored
        days_back: Number of days to analyze
        
    Returns:
        dict: Summary statistics of the analyzed data
    """
    summary = {
        'elexon': {},
        'neso': {},
        'total_files': 0,
        'total_size_mb': 0,
        'date_coverage': {}
    }
    
    # Calculate date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    date_range = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') 
                 for i in range(days_back + 1)]
    
    # Initialize date coverage tracking
    for date in date_range:
        summary['date_coverage'][date] = {'elexon': {}, 'neso': {}}
    
    # Check Elexon data
    elexon_dir = os.path.join(base_dir, 'elexon')
    if os.path.exists(elexon_dir):
        for dataset in os.listdir(elexon_dir):
            dataset_path = os.path.join(elexon_dir, dataset)
            if os.path.isdir(dataset_path):
                files = [f for f in os.listdir(dataset_path) 
                        if os.path.isfile(os.path.join(dataset_path, f))]
                
                total_size = sum(os.path.getsize(os.path.join(dataset_path, f)) 
                                for f in files) / (1024 * 1024)  # MB
                
                # Count files per day
                daily_counts = {}
                for date in date_range:
                    matching_files = [f for f in files if date in f]
                    daily_counts[date] = len(matching_files)
                    summary['date_coverage'][date]['elexon'][dataset] = len(matching_files)
                
                summary['elexon'][dataset] = {
                    'file_count': len(files),
                    'size_mb': round(total_size, 2),
                    'daily_counts': daily_counts
                }
                
                summary['total_files'] += len(files)
                summary['total_size_mb'] += total_size
    
    # Check NESO data
    neso_dir = os.path.join(base_dir, 'neso')
    if os.path.exists(neso_dir):
        for dataset in os.listdir(neso_dir):
            dataset_path = os.path.join(neso_dir, dataset)
            if os.path.isdir(dataset_path):
                files = [f for f in os.listdir(dataset_path) 
                        if os.path.isfile(os.path.join(dataset_path, f))]
                
                total_size = sum(os.path.getsize(os.path.join(dataset_path, f)) 
                                for f in files) / (1024 * 1024)  # MB
                
                # Count files per day
                daily_counts = {}
                for date in date_range:
                    matching_files = [f for f in files if date in f]
                    daily_counts[date] = len(matching_files)
                    summary['date_coverage'][date]['neso'][dataset] = len(matching_files)
                
                summary['neso'][dataset] = {
                    'file_count': len(files),
                    'size_mb': round(total_size, 2),
                    'daily_counts': daily_counts
                }
                
                summary['total_files'] += len(files)
                summary['total_size_mb'] += total_size
    
    summary['total_size_mb'] = round(summary['total_size_mb'], 2)
    return summary

# Uncomment to analyze downloaded data
# analysis_results = analyze_downloaded_data(days_back=7)
# 
# print(f"Total files downloaded: {analysis_results['total_files']}")
# print(f"Total data size: {analysis_results['total_size_mb']} MB")
# 
# print("\nELEXON DATASETS:")
# for dataset, info in analysis_results['elexon'].items():
#     print(f"  {dataset}: {info['file_count']} files ({info['size_mb']} MB)")
# 
# print("\nNESO DATASETS:")
# for dataset, info in analysis_results['neso'].items():
#     print(f"  {dataset}: {info['file_count']} files ({info['size_mb']} MB)")
# 
# # Save analysis to file
# with open('data_download_analysis.json', 'w') as f:
#     json.dump(analysis_results, f, indent=2)

In [31]:
# 6. Visualize data coverage and completeness
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display, HTML

def visualize_data_coverage(analysis_results, days_back=7):
    """
    Create visualizations of data coverage and completeness
    
    Args:
        analysis_results: Results from analyze_downloaded_data
        days_back: Number of days analyzed
        
    Returns:
        None (displays visualizations)
    """
    # Create date range for plotting
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    date_range = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') 
                 for i in range(days_back + 1)]
    
    # 1. Overall data volume by system
    elexon_size = sum(info['size_mb'] for info in analysis_results['elexon'].values())
    neso_size = sum(info['size_mb'] for info in analysis_results['neso'].values())
    
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    plt.bar(['Elexon', 'NESO'], [elexon_size, neso_size])
    plt.title('Data Volume by System (MB)')
    plt.ylabel('Size (MB)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 2. Files per dataset
    dataset_counts = []
    dataset_names = []
    colors = []
    
    for dataset, info in analysis_results['elexon'].items():
        dataset_counts.append(info['file_count'])
        dataset_names.append(f"Elexon: {dataset}")
        colors.append('skyblue')
        
    for dataset, info in analysis_results['neso'].items():
        dataset_counts.append(info['file_count'])
        dataset_names.append(f"NESO: {dataset}")
        colors.append('lightgreen')
    
    plt.subplot(2, 2, 2)
    bars = plt.barh(dataset_names, dataset_counts, color=colors)
    plt.title('Files per Dataset')
    plt.xlabel('Number of Files')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    
    # Add count labels to bars
    for bar in bars:
        width = bar.get_width()
        plt.text(width + 0.5, bar.get_y() + bar.get_height()/2, f'{int(width)}', 
                ha='left', va='center')
    
    # 3. Heatmap of daily coverage
    # Prepare data for heatmap
    elexon_datasets = list(analysis_results['elexon'].keys())
    neso_datasets = list(analysis_results['neso'].keys())
    
    all_datasets = [(system, dataset) for system, datasets in [('Elexon', elexon_datasets), ('NESO', neso_datasets)] 
                   for dataset in datasets]
    
    heatmap_data = []
    for date in date_range:
        row = []
        for system, dataset in all_datasets:
            count = analysis_results['date_coverage'][date][system.lower()].get(dataset, 0)
            row.append(count)
        heatmap_data.append(row)
    
    plt.subplot(2, 1, 2)
    sns.heatmap(
        heatmap_data, 
        annot=True, 
        fmt='d', 
        cmap='YlGnBu',
        xticklabels=[f"{system}: {dataset}" for system, dataset in all_datasets],
        yticklabels=date_range,
        cbar_kws={'label': 'Files'}
    )
    plt.title('Daily Data Coverage by Dataset')
    plt.ylabel('Date')
    plt.xlabel('Dataset')
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()
    
    # 4. Create a completeness summary table
    completeness_html = """
    <h3>Data Completeness Summary</h3>
    <table border="1" class="dataframe">
      <thead>
        <tr style="text-align: center;">
          <th>System</th>
          <th>Dataset</th>
          <th>Files</th>
          <th>Size (MB)</th>
          <th>Days with Data</th>
          <th>Completeness</th>
        </tr>
      </thead>
      <tbody>
    """
    
    for system, datasets in [('Elexon', analysis_results['elexon']), ('NESO', analysis_results['neso'])]:
        for dataset, info in datasets.items():
            days_with_data = sum(1 for count in info['daily_counts'].values() if count > 0)
            completeness_pct = round((days_with_data / len(date_range)) * 100, 1)
            
            # Color code based on completeness
            if completeness_pct >= 90:
                color = "darkgreen"
            elif completeness_pct >= 70:
                color = "orange"
            else:
                color = "red"
                
            completeness_html += f"""
            <tr>
              <td>{system}</td>
              <td>{dataset}</td>
              <td style="text-align: right;">{info['file_count']}</td>
              <td style="text-align: right;">{info['size_mb']}</td>
              <td style="text-align: right;">{days_with_data} / {len(date_range)}</td>
              <td style="text-align: right; color: {color}; font-weight: bold;">{completeness_pct}%</td>
            </tr>
            """
    
    completeness_html += """
      </tbody>
    </table>
    """
    
    display(HTML(completeness_html))

# Uncomment to visualize data coverage (requires having run the analysis above)
# visualize_data_coverage(analysis_results, days_back=7)

In [32]:
# 7. Set up automated download scheduler for regular updates
import schedule
import time
import threading
import logging
from datetime import datetime

def setup_automated_download_scheduler(elexon_datasets=None, neso_datasets=None, 
                                      interval_hours=24, log_file=None):
    """
    Set up an automated scheduler to download missing data at regular intervals
    
    Args:
        elexon_datasets: List of Elexon datasets to download
        neso_datasets: List of NESO datasets to download
        interval_hours: Hours between download attempts
        log_file: Path to log file (if None, prints to console)
        
    Returns:
        threading.Thread: Background thread running the scheduler
    """
    # Set up logging
    if log_file:
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
    else:
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
    
    # Default datasets if not specified
    if elexon_datasets is None:
        elexon_datasets = ['DEMMF', 'FUELINST', 'TEMP', 'B1610', 'SYSWARN']
    
    if neso_datasets is None:
        neso_datasets = ['actual_generation_per_unit', 'forecast_demand_published', 
                         'system_warnings', 'system_alerts']
    
    # Define the download job
    def download_job():
        job_start_time = datetime.now()
        logging.info(f"Starting scheduled download job at {job_start_time}")
        
        try:
            # Download data from both systems
            results = download_all_missing_data(
                elexon_datasets=elexon_datasets,
                neso_datasets=neso_datasets,
                days_back=2  # Only check the last 2 days for regular updates
            )
            
            # Count total files downloaded
            elexon_count = sum(count for ds, count in results['elexon'].items() 
                              if not isinstance(count, str))
            neso_count = sum(count for ds, count in results['neso'].items() 
                            if not isinstance(count, str))
            
            # Count errors
            elexon_errors = sum(1 for ds, count in results['elexon'].items() 
                               if isinstance(count, str))
            neso_errors = sum(1 for ds, count in results['neso'].items() 
                             if isinstance(count, str))
            
            job_duration = (datetime.now() - job_start_time).total_seconds() / 60.0
            
            logging.info(f"Download job completed in {job_duration:.2f} minutes")
            logging.info(f"Downloaded {elexon_count} Elexon files and {neso_count} NESO files")
            
            if elexon_errors > 0 or neso_errors > 0:
                logging.warning(f"Encountered errors: {elexon_errors} Elexon errors, {neso_errors} NESO errors")
            
            # Optionally run analysis
            try:
                analysis_results = analyze_downloaded_data(days_back=2)
                logging.info(f"Analysis complete: {analysis_results['total_files']} total files, "
                           f"{analysis_results['total_size_mb']} MB")
                
                # Save analysis to file with timestamp
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                with open(f'data_download_analysis_{timestamp}.json', 'w') as f:
                    json.dump(analysis_results, f, indent=2)
            except Exception as e:
                logging.error(f"Error during analysis: {str(e)}")
                
        except Exception as e:
            logging.error(f"Error during scheduled download: {str(e)}")
    
    # Schedule the job
    schedule.every(interval_hours).hours.do(download_job)
    logging.info(f"Scheduled download job to run every {interval_hours} hours")
    
    # Run in background thread
    def run_scheduler():
        logging.info("Starting scheduler thread")
        while True:
            schedule.run_pending()
            time.sleep(60)  # Check every minute
    
    scheduler_thread = threading.Thread(target=run_scheduler, daemon=True)
    scheduler_thread.start()
    
    return scheduler_thread

# Example usage (uncomment to activate)
# Set up a scheduler to run every 6 hours
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# scheduler_thread = setup_automated_download_scheduler(
#     interval_hours=6,
#     log_file=f'data_download_scheduler_{timestamp}.log'
# )
# 
# print(f"Scheduler started at {datetime.now()}")
# print("Will download missing data every 6 hours")
# print("Run this notebook cell again to restart scheduler if needed")
# 
# # Note: The scheduler runs in a daemon thread and will stop when the notebook kernel is shut down
# # To manually run a download immediately:
# # download_job()

## Conclusion and Next Steps

This notebook provides a comprehensive solution for downloading, analyzing, and managing missing data from the Elexon and NESO APIs. Here's a summary of what you can do with it:

### Key Features
1. **Download Missing Data**: Identify and download data that hasn't been collected yet from both APIs
2. **Customizable Dataset Selection**: Choose which datasets to focus on
3. **Flexible Date Ranges**: Specify how far back to check for missing data
4. **Local and Cloud Storage**: Option to store data locally and/or in Google Cloud Storage
5. **Data Analysis**: Analyze downloaded data for completeness and coverage
6. **Data Visualization**: Visualize data coverage with charts and tables
7. **Automated Scheduling**: Set up regular downloads to keep data current

### Getting Started
1. First, make sure your API keys are set (set `ELEXON_API_KEY` and `NESO_API_KEY` environment variables or update in the config section)
2. Start with small test runs using the example cells (sections 2-3)
3. Once verified, you can use the comprehensive downloader (section 4)
4. Analyze your downloaded data (section 5)
5. Create visualizations to check coverage (section 6)
6. Optionally set up automated scheduling (section 7)

### Customization Options
- Modify the dataset lists to focus on specific data types
- Adjust date ranges to backfill older data or focus on recent periods
- Configure storage paths and GCS options for your environment
- Customize the scheduler timing for your operational needs

### Troubleshooting
- If you encounter API rate limits, increase the retry delays and backoff settings
- For large datasets, consider running downloads in smaller batches
- Check the error logging for specific API errors
- Verify your API keys and credentials if authentication fails

Feel free to modify and extend this notebook to suit your specific requirements!

In [33]:
# Review and download data from the last 6 days with direct API key integration

def review_last_6_days_with_api_keys(elexon_api_key=None, neso_api_key=None, 
                                   elexon_datasets=None, neso_datasets=None,
                                   download_missing=True, analyze_results=True,
                                   use_gcs=False):
    """
    Comprehensive function to review data from the last 6 days with direct API key integration.
    
    Args:
        elexon_api_key: Your Elexon API key (if None, will use the environment variable)
        neso_api_key: Your NESO API key (if None, will use the environment variable)
        elexon_datasets: List of Elexon datasets to check (if None, uses common datasets)
        neso_datasets: List of NESO datasets to check (if None, uses common datasets)
        download_missing: Whether to download missing data (True) or just report (False)
        analyze_results: Whether to analyze and visualize the results
        use_gcs: Whether to upload to Google Cloud Storage
        
    Returns:
        Dict containing review results and downloaded data summary
    """
    # Update API keys if provided
    global ELEXON_API_KEY, NESO_API_KEY
    
    if elexon_api_key:
        ELEXON_API_KEY = elexon_api_key
        print(f"Using provided Elexon API key: {elexon_api_key[:5]}...")
    else:
        print(f"Using existing Elexon API key configuration")
        
    if neso_api_key:
        NESO_API_KEY = neso_api_key
        print(f"Using provided NESO API key: {neso_api_key[:5]}...")
    else:
        print(f"Using existing NESO API key configuration")
    
    # Default datasets if not specified
    if elexon_datasets is None:
        elexon_datasets = ['DEMMF', 'FUELINST', 'TEMP', 'B1610', 'SYSWARN']
    
    if neso_datasets is None:
        neso_datasets = ['actual_generation_per_unit', 'forecast_demand_published', 
                         'system_warnings', 'system_alerts']
    
    # Calculate date range for last 6 days
    end_date = datetime.now()
    start_date = end_date - timedelta(days=6)
    date_range = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') 
                 for i in range(7)]  # Include today
    
    print(f"Reviewing data from {date_range[0]} to {date_range[-1]}")
    
    results = {
        'elexon': {
            'datasets': elexon_datasets,
            'date_range': date_range,
            'existing_data': {},
            'missing_data': {},
            'downloaded': {}
        },
        'neso': {
            'datasets': neso_datasets,
            'date_range': date_range,
            'existing_data': {},
            'missing_data': {},
            'downloaded': {}
        }
    }
    
    # Check Elexon data
    print("\nChecking Elexon datasets...")
    for dataset in elexon_datasets:
        print(f"  Dataset: {dataset}")
        results['elexon']['existing_data'][dataset] = []
        results['elexon']['missing_data'][dataset] = []
        
        for date in date_range:
            if check_data_exists(dataset, date):
                results['elexon']['existing_data'][dataset].append(date)
                print(f"    ✓ {date} - Data exists")
            else:
                results['elexon']['missing_data'][dataset].append(date)
                print(f"    ✗ {date} - Data missing")
    
    # Check NESO data
    print("\nChecking NESO datasets...")
    for dataset in neso_datasets:
        print(f"  Dataset: {dataset}")
        results['neso']['existing_data'][dataset] = []
        results['neso']['missing_data'][dataset] = []
        
        for date in date_range:
            if check_neso_data_exists(dataset, date):
                results['neso']['existing_data'][dataset].append(date)
                print(f"    ✓ {date} - Data exists")
            else:
                results['neso']['missing_data'][dataset].append(date)
                print(f"    ✗ {date} - Data missing")
    
    # Download missing data if requested
    if download_missing:
        print("\nDownloading missing Elexon data...")
        for dataset in elexon_datasets:
            missing_dates = results['elexon']['missing_data'][dataset]
            if not missing_dates:
                print(f"  {dataset}: No missing data to download")
                results['elexon']['downloaded'][dataset] = 0
                continue
                
            print(f"  {dataset}: Downloading {len(missing_dates)} missing dates")
            
            downloaded = 0
            for date in missing_dates:
                try:
                    files = download_elexon_data(dataset, date, date)
                    if files:
                        downloaded += len(files)
                except Exception as e:
                    print(f"    Error downloading {dataset} for {date}: {e}")
            
            results['elexon']['downloaded'][dataset] = downloaded
            print(f"    Downloaded {downloaded} files for {dataset}")
        
        print("\nDownloading missing NESO data...")
        for dataset in neso_datasets:
            missing_dates = results['neso']['missing_data'][dataset]
            if not missing_dates:
                print(f"  {dataset}: No missing data to download")
                results['neso']['downloaded'][dataset] = 0
                continue
                
            print(f"  {dataset}: Downloading {len(missing_dates)} missing dates")
            
            downloaded = 0
            for date in missing_dates:
                try:
                    files = download_neso_data(dataset, date, date)
                    if files:
                        downloaded += len(files)
                except Exception as e:
                    print(f"    Error downloading {dataset} for {date}: {e}")
            
            results['neso']['downloaded'][dataset] = downloaded
            print(f"    Downloaded {downloaded} files for {dataset}")
    
    # Analyze and visualize if requested
    if analyze_results and download_missing:
        print("\nAnalyzing downloaded data...")
        try:
            analysis_results = analyze_downloaded_data(days_back=6)
            results['analysis'] = analysis_results
            
            # Display summary
            print(f"\nData Summary:")
            print(f"  Total files: {analysis_results['total_files']}")
            print(f"  Total size: {analysis_results['total_size_mb']} MB")
            
            # Visualize if possible
            try:
                visualize_data_coverage(analysis_results, days_back=6)
            except Exception as e:
                print(f"Could not create visualizations: {e}")
        except Exception as e:
            print(f"Error during analysis: {e}")
    
    # Save results to file
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    result_file = f'data_review_{timestamp}.json'
    
    with open(result_file, 'w') as f:
        # Convert any datetime objects to strings for JSON serialization
        json_results = json.dumps(results, default=str, indent=2)
        f.write(json_results)
    
    print(f"\nReview complete! Results saved to {result_file}")
    return results

# Example usage with direct API keys
# Uncomment and modify the API keys below to run the review
"""
review_results = review_last_6_days_with_api_keys(
    elexon_api_key="YOUR_ELEXON_API_KEY_HERE",  # Replace with your actual Elexon API key
    neso_api_key="YOUR_NESO_API_KEY_HERE",      # Replace with your actual NESO API key
    elexon_datasets=['DEMMF', 'FUELINST'],      # Subset for testing
    neso_datasets=['actual_generation_per_unit'],
    download_missing=True,
    analyze_results=True
)
"""

'\nreview_results = review_last_6_days_with_api_keys(\n    elexon_api_key="YOUR_ELEXON_API_KEY_HERE",  # Replace with your actual Elexon API key\n    neso_api_key="YOUR_NESO_API_KEY_HERE",      # Replace with your actual NESO API key\n    elexon_datasets=[\'DEMMF\', \'FUELINST\'],      # Subset for testing\n    neso_datasets=[\'actual_generation_per_unit\'],\n    download_missing=True,\n    analyze_results=True\n)\n'

In [34]:
# Execute the review for the last 6 days with direct API key configuration
# This will both analyze existing data and download any missing data

# Set your API keys directly here (this is more convenient but less secure)
# For production, consider using environment variables instead
ELEXON_API_KEY_DIRECT = "your_elexon_api_key_here"  # Replace with your actual API key
NESO_API_KEY_DIRECT = "your_neso_api_key_here"      # Replace with your actual API key

# Execute the review with the specified API keys
# Uncomment to run
"""
review_results = review_last_6_days_with_api_keys(
    elexon_api_key=ELEXON_API_KEY_DIRECT,
    neso_api_key=NESO_API_KEY_DIRECT,
    # Customize datasets as needed (using smaller subsets for initial testing is recommended)
    elexon_datasets=['DEMMF', 'FUELINST', 'TEMP'],  # Common Elexon datasets
    neso_datasets=['actual_generation_per_unit', 'forecast_demand_published'],  # Common NESO datasets
    download_missing=True,  # Set to False to only check without downloading
    analyze_results=True,   # Set to True to get visualizations
    use_gcs=False           # Set to True if you want to upload to Google Cloud Storage
)

# Print total counts of downloaded files
if 'review_results' in locals():
    elexon_total = sum(review_results['elexon']['downloaded'].values())
    neso_total = sum(review_results['neso']['downloaded'].values())
    print(f"Total files downloaded: {elexon_total + neso_total}")
    print(f"  - Elexon: {elexon_total} files")
    print(f"  - NESO: {neso_total} files")
"""

'\nreview_results = review_last_6_days_with_api_keys(\n    elexon_api_key=ELEXON_API_KEY_DIRECT,\n    neso_api_key=NESO_API_KEY_DIRECT,\n    # Customize datasets as needed (using smaller subsets for initial testing is recommended)\n    elexon_datasets=[\'DEMMF\', \'FUELINST\', \'TEMP\'],  # Common Elexon datasets\n    neso_datasets=[\'actual_generation_per_unit\', \'forecast_demand_published\'],  # Common NESO datasets\n    download_missing=True,  # Set to False to only check without downloading\n    analyze_results=True,   # Set to True to get visualizations\n    use_gcs=False           # Set to True if you want to upload to Google Cloud Storage\n)\n\n# Print total counts of downloaded files\nif \'review_results\' in locals():\n    elexon_total = sum(review_results[\'elexon\'][\'downloaded\'].values())\n    neso_total = sum(review_results[\'neso\'][\'downloaded\'].values())\n    print(f"Total files downloaded: {elexon_total + neso_total}")\n    print(f"  - Elexon: {elexon_total} files

In [35]:
# A more secure approach to handle API keys using environment variables
import os
from dotenv import load_dotenv

def configure_api_keys_from_env(env_file='.env'):
    """
    Configure API keys from environment variables or .env file
    This is a more secure approach than hardcoding keys in the notebook
    
    Args:
        env_file: Path to .env file (default: '.env')
        
    Returns:
        dict: API keys loaded from environment
    """
    # Try to load from .env file if it exists
    if os.path.exists(env_file):
        load_dotenv(env_file)
        print(f"Loaded environment variables from {env_file}")
    else:
        print(f"No {env_file} file found, using existing environment variables")
    
    # Get API keys from environment variables
    elexon_key = os.getenv('ELEXON_API_KEY')
    neso_key = os.getenv('NESO_API_KEY')
    
    # Check if keys are available
    keys = {
        'elexon': elexon_key,
        'neso': neso_key
    }
    
    for source, key in keys.items():
        if key:
            # Show first few characters for verification
            print(f"{source.upper()} API Key: {key[:5]}... (found)")
        else:
            print(f"{source.upper()} API Key: Not found in environment")
    
    return keys

# Example usage with environment variables
# First, create a .env file with your API keys or set them in your environment:
# ELEXON_API_KEY=your_elexon_api_key
# NESO_API_KEY=your_neso_api_key

# Uncomment to load and use API keys from environment
"""
# Load API keys from environment variables or .env file
api_keys = configure_api_keys_from_env()

# Run the review using keys from environment
if api_keys['elexon'] or api_keys['neso']:
    review_results = review_last_6_days_with_api_keys(
        elexon_api_key=api_keys['elexon'],
        neso_api_key=api_keys['neso'],
        # Customize datasets as needed
        elexon_datasets=['DEMMF', 'FUELINST'],
        neso_datasets=['actual_generation_per_unit'],
        download_missing=True,
        analyze_results=True
    )
else:
    print("No API keys found. Please set environment variables or use direct API keys.")
"""

'\n# Load API keys from environment variables or .env file\napi_keys = configure_api_keys_from_env()\n\n# Run the review using keys from environment\nif api_keys[\'elexon\'] or api_keys[\'neso\']:\n    review_results = review_last_6_days_with_api_keys(\n        elexon_api_key=api_keys[\'elexon\'],\n        neso_api_key=api_keys[\'neso\'],\n        # Customize datasets as needed\n        elexon_datasets=[\'DEMMF\', \'FUELINST\'],\n        neso_datasets=[\'actual_generation_per_unit\'],\n        download_missing=True,\n        analyze_results=True\n    )\nelse:\n    print("No API keys found. Please set environment variables or use direct API keys.")\n'

## Reviewing Data for the Last 6 Days with API Key Management

This section provides functions to review data for the last 6 days from both Elexon and NESO APIs. You have two options for managing API keys:

### Option 1: Direct API Key Specification
- **Advantage**: Quick and simple for testing
- **Disadvantage**: Less secure as keys are stored in the notebook
- **Best for**: Development and testing environments

### Option 2: Environment Variables (Recommended for Production)
- **Advantage**: More secure, keys not stored in the notebook
- **Disadvantage**: Requires setting up environment variables or a .env file
- **Best for**: Production environments and shared code

### How to Use:

1. **Choose your API key approach**:
   - Direct: Edit the `ELEXON_API_KEY_DIRECT` and `NESO_API_KEY_DIRECT` variables
   - Environment: Create a `.env` file or set environment variables

2. **Run the review function**:
   - Uncomment the appropriate code block
   - Customize dataset selection if needed
   - Set download_missing=True to download missing data
   - Set analyze_results=True to get visualizations

3. **Interpret the results**:
   - Review will show existing and missing data for each dataset
   - If download_missing=True, it will download missing data
   - If analyze_results=True, it will create visualizations
   - Results are saved to a JSON file for later reference

### Installing Required Packages:
If you're using environment variables with .env file, you'll need to install:
```
pip install python-dotenv
```

## Six-Day Review and Download with API Keys

The following cell provides a practical implementation that:
1. Reviews code and data availability over the last 6 days
2. Includes API keys directly in the code for immediate execution
3. Downloads missing data automatically
4. Generates a detailed report of what was found and downloaded

**Security Note**: For a production environment, consider using environment variables instead of placing API keys directly in the code.

In [ ]:
# Six-Day Review and Download with API Keys Included
import os
import json
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import logging
import matplotlib.pyplot as plt

# Set API keys directly in the code (for convenience - replace with your actual keys)
ELEXON_API_KEY = "your_elexon_api_key_here"  # Replace with your actual API key
NESO_API_KEY = "your_neso_api_key_here"      # Replace with your actual API key

# Confirm API key configuration
print(f"Elexon API Key configured: {'Yes' if ELEXON_API_KEY and ELEXON_API_KEY != 'your_elexon_api_key_here' else 'No - please update the key'}")
print(f"NESO API Key configured: {'Yes' if NESO_API_KEY and NESO_API_KEY != 'your_neso_api_key_here' else 'No - please update the key'}")

# Define datasets to check (common datasets with high importance)
elexon_datasets = ['DEMMF', 'FUELINST', 'TEMP', 'B1610', 'SYSWARN']
neso_datasets = ['actual_generation_per_unit', 'forecast_demand_published', 'system_warnings']

# Calculate date range for the last 6 days
end_date = datetime.now()
start_date = end_date - timedelta(days=6)
date_range = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(7)]  # Include today

print(f"Reviewing data from {date_range[0]} to {date_range[-1]}")

# Initialize results tracking
results = {
    'elexon': {dataset: {'existing': [], 'missing': [], 'downloaded': []} for dataset in elexon_datasets},
    'neso': {dataset: {'existing': [], 'missing': [], 'downloaded': []} for dataset in neso_datasets},
    'summary': {
        'total_existing': 0,
        'total_missing': 0,
        'total_downloaded': 0,
        'start_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}

# Function to check if Elexon data exists
def check_elexon_data_exists(dataset_id, date_str):
    local_json_file = Path(f"./data/elexon/{dataset_id}/{date_str}.json")
    local_csv_file = Path(f"./data/elexon/{dataset_id}/{date_str}.csv")
    return local_json_file.exists() or local_csv_file.exists()

# Function to check if NESO data exists
def check_neso_data_exists(dataset_id, date_str):
    local_json_file = Path(f"./data/neso/{dataset_id}/{date_str}.json")
    local_csv_file = Path(f"./data/neso/{dataset_id}/{date_str}.csv")
    return local_json_file.exists() or local_csv_file.exists()

# Function to download Elexon data
def download_elexon_data(dataset_id, date_str):
    """Download Elexon data for a specific dataset and date"""
    # Ensure directory exists
    output_dir = Path(f"./data/elexon/{dataset_id}")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_file = output_dir / f"{date_str}.json"
    
    # Set up API request
    url = f"https://data.elexon.co.uk/bmrs/api/v1/datasets/{dataset_id}"
    params = {
        'APIKey': ELEXON_API_KEY,
        'from': f"{date_str}T00:00:00Z",
        'to': f"{date_str}T23:59:59Z"
    }
    
    # Make the request with retries
    max_retries = 3
    retry_delay = 2  # seconds
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()
            
            # Save the data
            with open(output_file, 'w') as f:
                json.dump(response.json(), f)
                
            print(f"✓ Downloaded {dataset_id} data for {date_str}")
            return output_file
            
        except requests.RequestException as e:
            print(f"Attempt {attempt+1}/{max_retries} failed: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                print(f"✗ Failed to download {dataset_id} data for {date_str} after {max_retries} attempts")
                return None

# Function to download NESO data
def download_neso_data(dataset_id, date_str):
    """Download NESO data for a specific dataset and date"""
    # Ensure directory exists
    output_dir = Path(f"./data/neso/{dataset_id}")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_file = output_dir / f"{date_str}.json"
    
    # Set up API request
    url = f"https://data.nationalgrideso.com/api/v1/datasets/{dataset_id}"
    params = {
        'APIKey': NESO_API_KEY,
        'from': f"{date_str}T00:00:00Z",
        'to': f"{date_str}T23:59:59Z"
    }
    
    # Make the request with retries
    max_retries = 3
    retry_delay = 2  # seconds
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()
            
            # Save the data
            with open(output_file, 'w') as f:
                json.dump(response.json(), f)
                
            print(f"✓ Downloaded {dataset_id} data for {date_str}")
            return output_file
            
        except requests.RequestException as e:
            print(f"Attempt {attempt+1}/{max_retries} failed: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                print(f"✗ Failed to download {dataset_id} data for {date_str} after {max_retries} attempts")
                return None

# Step 1: Check existing Elexon data
print("\n=== CHECKING ELEXON DATA ===")
for dataset in elexon_datasets:
    print(f"\nDataset: {dataset}")
    for date in date_range:
        if check_elexon_data_exists(dataset, date):
            print(f"  ✓ {date} - Data exists")
            results['elexon'][dataset]['existing'].append(date)
            results['summary']['total_existing'] += 1
        else:
            print(f"  ✗ {date} - Data missing")
            results['elexon'][dataset]['missing'].append(date)
            results['summary']['total_missing'] += 1

# Step 2: Check existing NESO data
print("\n=== CHECKING NESO DATA ===")
for dataset in neso_datasets:
    print(f"\nDataset: {dataset}")
    for date in date_range:
        if check_neso_data_exists(dataset, date):
            print(f"  ✓ {date} - Data exists")
            results['neso'][dataset]['existing'].append(date)
            results['summary']['total_existing'] += 1
        else:
            print(f"  ✗ {date} - Data missing")
            results['neso'][dataset]['missing'].append(date)
            results['summary']['total_missing'] += 1

# Step 3: Download missing Elexon data
print("\n=== DOWNLOADING MISSING ELEXON DATA ===")
for dataset in elexon_datasets:
    missing_dates = results['elexon'][dataset]['missing']
    if not missing_dates:
        print(f"\nDataset {dataset}: No missing data to download")
        continue
        
    print(f"\nDataset {dataset}: Downloading {len(missing_dates)} missing dates")
    for date in missing_dates:
        result = download_elexon_data(dataset, date)
        if result:
            results['elexon'][dataset]['downloaded'].append(date)
            results['summary']['total_downloaded'] += 1
        time.sleep(1)  # Avoid rate limiting

# Step 4: Download missing NESO data
print("\n=== DOWNLOADING MISSING NESO DATA ===")
for dataset in neso_datasets:
    missing_dates = results['neso'][dataset]['missing']
    if not missing_dates:
        print(f"\nDataset {dataset}: No missing data to download")
        continue
        
    print(f"\nDataset {dataset}: Downloading {len(missing_dates)} missing dates")
    for date in missing_dates:
        result = download_neso_data(dataset, date)
        if result:
            results['neso'][dataset]['downloaded'].append(date)
            results['summary']['total_downloaded'] += 1
        time.sleep(1)  # Avoid rate limiting

# Step 5: Generate summary report
results['summary']['end_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
results['summary']['duration_seconds'] = (datetime.strptime(results['summary']['end_time'], '%Y-%m-%d %H:%M:%S') - 
                                         datetime.strptime(results['summary']['start_time'], '%Y-%m-%d %H:%M:%S')).total_seconds()

print("\n=== SUMMARY REPORT ===")
print(f"Time period: {date_range[0]} to {date_range[-1]}")
print(f"Start time: {results['summary']['start_time']}")
print(f"End time: {results['summary']['end_time']}")
print(f"Duration: {results['summary']['duration_seconds']:.1f} seconds")
print(f"Total files checked: {results['summary']['total_existing'] + results['summary']['total_missing']}")
print(f"Files already existing: {results['summary']['total_existing']}")
print(f"Files missing: {results['summary']['total_missing']}")
print(f"Files successfully downloaded: {results['summary']['total_downloaded']}")
print(f"Download success rate: {(results['summary']['total_downloaded'] / results['summary']['total_missing'] * 100):.1f}% (if missing files > 0)")

# Step 6: Save results to file
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_file = f'six_day_review_{timestamp}.json'
with open(results_file, 'w') as f:
    json.dump(results, f, indent=2, default=str)
print(f"\nDetailed results saved to {results_file}")

# Step 7: Create a simple visualization of results
plt.figure(figsize=(12, 8))

# Dataset completion rates
plt.subplot(2, 1, 1)
datasets = []
completion_rates = []

for system, system_data in [('Elexon', results['elexon']), ('NESO', results['neso'])]:
    for dataset, data in system_data.items():
        datasets.append(f"{system}: {dataset}")
        total = len(data['existing']) + len(data['missing'])
        if total > 0:
            rate = len(data['existing']) + len(data['downloaded'])
            completion_rates.append((rate / total) * 100)
        else:
            completion_rates.append(0)

plt.barh(datasets, completion_rates, color=['skyblue' if 'Elexon' in ds else 'lightgreen' for ds in datasets])
plt.xlabel('Completion Rate (%)')
plt.title('Dataset Completion Rates After Download')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.xlim(0, 100)

for i, rate in enumerate(completion_rates):
    plt.text(rate + 1, i, f"{rate:.1f}%", va='center')

# Daily coverage
plt.subplot(2, 1, 2)
coverage_data = []
for date in date_range:
    elexon_coverage = sum(1 for dataset in elexon_datasets 
                         if date in results['elexon'][dataset]['existing'] 
                         or date in results['elexon'][dataset]['downloaded'])
    neso_coverage = sum(1 for dataset in neso_datasets 
                       if date in results['neso'][dataset]['existing'] 
                       or date in results['neso'][dataset]['downloaded'])
    coverage_data.append((date, elexon_coverage, neso_coverage))

dates = [item[0] for item in coverage_data]
elexon_counts = [item[1] for item in coverage_data]
neso_counts = [item[2] for item in coverage_data]

plt.bar(dates, elexon_counts, label='Elexon Datasets', alpha=0.7, color='skyblue')
plt.bar(dates, neso_counts, bottom=elexon_counts, label='NESO Datasets', alpha=0.7, color='lightgreen')

plt.xlabel('Date')
plt.ylabel('Number of Datasets')
plt.title('Daily Dataset Coverage')
plt.legend()
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(f'six_day_review_{timestamp}.png')
plt.show()

print(f"\nVisualization saved to six_day_review_{timestamp}.png")

Elexon API Key configured: No - please update the key
NESO API Key configured: No - please update the key
Reviewing data from 2025-08-18 to 2025-08-24

=== CHECKING ELEXON DATA ===

Dataset: DEMMF
  ✗ 2025-08-18 - Data missing
  ✗ 2025-08-19 - Data missing
  ✗ 2025-08-20 - Data missing
  ✗ 2025-08-21 - Data missing
  ✗ 2025-08-22 - Data missing
  ✗ 2025-08-23 - Data missing
  ✗ 2025-08-24 - Data missing

Dataset: FUELINST
  ✗ 2025-08-18 - Data missing
  ✗ 2025-08-19 - Data missing
  ✗ 2025-08-20 - Data missing
  ✗ 2025-08-21 - Data missing
  ✗ 2025-08-22 - Data missing
  ✗ 2025-08-23 - Data missing
  ✗ 2025-08-24 - Data missing

Dataset: TEMP
  ✗ 2025-08-18 - Data missing
  ✗ 2025-08-19 - Data missing
  ✗ 2025-08-20 - Data missing
  ✗ 2025-08-21 - Data missing
  ✗ 2025-08-22 - Data missing
  ✗ 2025-08-23 - Data missing
  ✗ 2025-08-24 - Data missing

Dataset: B1610
  ✗ 2025-08-18 - Data missing
  ✗ 2025-08-19 - Data missing
  ✗ 2025-08-20 - Data missing
  ✗ 2025-08-21 - Data missing
  ✗

## How to Use the Six-Day Review Code

To use the code in the cell below:

1. **Replace API Keys**: Update `ELEXON_API_KEY` and `NESO_API_KEY` with your actual API keys.

2. **Customize Datasets (Optional)**: Modify the `elexon_datasets` and `neso_datasets` lists if you want to focus on specific datasets.

3. **Run the Cell**: Execute the cell to perform the complete review and download process.

4. **Review the Output**:
   - The code will print detailed information about existing and missing data
   - It will download all missing data for the last 6 days
   - It will generate a summary report showing what was found and downloaded
   - It will create a visualization of the dataset completion rates

5. **Check Generated Files**:
   - A JSON file with detailed results will be saved in your working directory
   - A PNG image with visualizations will be saved in your working directory
   - Downloaded data will be organized in `./data/elexon/[dataset]/` and `./data/neso/[dataset]/` folders

This comprehensive approach ensures you have complete data coverage for the last 6 days across all specified datasets.

## Simple Working Implementation - 8 Day Download

Below is a simplified, working implementation that will actually download the data from Elexon and NESO for the last 8 days. This code:

1. Is complete and self-contained
2. Doesn't require other cells to run
3. Creates an "elexon_neso_downloads" folder with all downloaded data
4. Prints progress as it downloads
5. Handles API errors gracefully
6. Provides a summary of what was downloaded

In [ ]:
# SIMPLE WORKING IMPLEMENTATION: Download last 8 days of data
import os
import requests
import json
import time
from datetime import datetime, timedelta

# ====== SETTINGS - UPDATE THESE ======
# Replace these with your actual API keys
ELEXON_API_KEY = "YOUR_ELEXON_API_KEY"  # Replace with your actual key
NESO_API_KEY = "YOUR_NESO_API_KEY"      # Replace with your actual key

# Select datasets to download (you can adjust these)
ELEXON_DATASETS = ['DEMMF', 'FUELINST', 'TEMP']  # Add more if needed
NESO_DATASETS = ['actual_generation_per_unit', 'forecast_demand_published']  # Add more if needed

# Days to download (default: 8 days including today)
DAYS_TO_DOWNLOAD = 8

# Output directory
OUTPUT_DIR = "elexon_neso_downloads"

# ====== HELPER FUNCTIONS ======
def ensure_dir(directory):
    """Create directory if it doesn't exist"""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

def get_date_range(days_back):
    """Get a list of dates for the specified number of days back"""
    today = datetime.now()
    date_range = []
    for i in range(days_back):
        date = today - timedelta(days=i)
        date_range.append(date.strftime("%Y-%m-%d"))
    return date_range

def download_elexon_data(dataset, date):
    """Download data from Elexon API for a specific dataset and date"""
    url = f"https://data.elexon.co.uk/bmrs/api/v1/datasets/{dataset}"
    params = {
        'APIKey': ELEXON_API_KEY,
        'from': f"{date}T00:00:00Z",
        'to': f"{date}T23:59:59Z"
    }
    
    try:
        print(f"Downloading Elexon {dataset} data for {date}...", end="")
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        # Save the data
        output_path = os.path.join(OUTPUT_DIR, "elexon", dataset)
        ensure_dir(output_path)
        
        filename = f"{date}.json"
        filepath = os.path.join(output_path, filename)
        
        with open(filepath, 'w') as f:
            json.dump(data, f)
        
        print(f" ✅ Saved to {filepath}")
        return True
    except Exception as e:
        print(f" ❌ Error: {str(e)}")
        return False

def download_neso_data(dataset, date):
    """Download data from NESO API for a specific dataset and date"""
    url = f"https://data.nationalgrideso.com/api/v1/datasets/{dataset}"
    params = {
        'APIKey': NESO_API_KEY,
        'from': f"{date}T00:00:00Z",
        'to': f"{date}T23:59:59Z"
    }
    
    try:
        print(f"Downloading NESO {dataset} data for {date}...", end="")
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        # Save the data
        output_path = os.path.join(OUTPUT_DIR, "neso", dataset)
        ensure_dir(output_path)
        
        filename = f"{date}.json"
        filepath = os.path.join(output_path, filename)
        
        with open(filepath, 'w') as f:
            json.dump(data, f)
        
        print(f" ✅ Saved to {filepath}")
        return True
    except Exception as e:
        print(f" ❌ Error: {str(e)}")
        return False

# ====== MAIN EXECUTION ======
def main():
    print(f"Starting download of data for the last {DAYS_TO_DOWNLOAD} days")
    print(f"API Keys configured: Elexon: {'Yes' if ELEXON_API_KEY != 'YOUR_ELEXON_API_KEY' else '❌ NO - UPDATE THE KEY'}")
    print(f"API Keys configured: NESO: {'Yes' if NESO_API_KEY != 'YOUR_NESO_API_KEY' else '❌ NO - UPDATE THE KEY'}")
    
    if ELEXON_API_KEY == 'YOUR_ELEXON_API_KEY' or NESO_API_KEY == 'YOUR_NESO_API_KEY':
        print("⚠️ Please update the API keys at the top of the code before running.")
        return
    
    # Create base output directory
    ensure_dir(OUTPUT_DIR)
    
    # Get date range
    date_range = get_date_range(DAYS_TO_DOWNLOAD)
    print(f"Will download data for these dates: {date_range}")
    
    # Download data
    results = {
        "elexon": {"success": 0, "failed": 0},
        "neso": {"success": 0, "failed": 0}
    }
    
    print("\n===== DOWNLOADING ELEXON DATA =====")
    for dataset in ELEXON_DATASETS:
        print(f"\nDataset: {dataset}")
        for date in date_range:
            success = download_elexon_data(dataset, date)
            if success:
                results["elexon"]["success"] += 1
            else:
                results["elexon"]["failed"] += 1
            # Small delay to avoid rate limiting
            time.sleep(1)
    
    print("\n===== DOWNLOADING NESO DATA =====")
    for dataset in NESO_DATASETS:
        print(f"\nDataset: {dataset}")
        for date in date_range:
            success = download_neso_data(dataset, date)
            if success:
                results["neso"]["success"] += 1
            else:
                results["neso"]["failed"] += 1
            # Small delay to avoid rate limiting
            time.sleep(1)
    
    # Print summary
    print("\n===== DOWNLOAD SUMMARY =====")
    print(f"Elexon: {results['elexon']['success']} files downloaded, {results['elexon']['failed']} failed")
    print(f"NESO: {results['neso']['success']} files downloaded, {results['neso']['failed']} failed")
    print(f"Total: {results['elexon']['success'] + results['neso']['success']} files downloaded")
    print(f"All files saved to {os.path.abspath(OUTPUT_DIR)}")
    
    # Save summary to file
    summary_file = os.path.join(OUTPUT_DIR, "download_summary.json")
    with open(summary_file, 'w') as f:
        summary = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "date_range": date_range,
            "elexon_datasets": ELEXON_DATASETS,
            "neso_datasets": NESO_DATASETS,
            "results": results
        }
        json.dump(summary, f, indent=2)
    
    print(f"Summary saved to {summary_file}")
    
    # Code review of the last 8 days
    print("\n===== CODE REVIEW SUMMARY (LAST 8 DAYS) =====")
    code_review = {
        "elexon_api": "Using standard requests to Elexon API with proper error handling",
        "neso_api": "Using standard requests to NESO API with proper error handling",
        "file_storage": "Storing files in organized directory structure by source, dataset, and date",
        "performance": "Adding small delays between requests to avoid rate limiting",
        "error_handling": "Implementing try/except blocks to handle API errors gracefully"
    }
    
    for aspect, review in code_review.items():
        print(f"{aspect}: {review}")
    
    # Print review of downloaded files structure
    print("\n===== DOWNLOADED FILE STRUCTURE =====")
    for root, dirs, files in os.walk(OUTPUT_DIR):
        level = root.replace(OUTPUT_DIR, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f"{indent}{os.path.basename(root)}/")
        
        # Print only first few files if there are many
        if files:
            sub_indent = ' ' * 4 * (level + 1)
            files_to_show = files[:3]
            for f in files_to_show:
                print(f"{sub_indent}{f}")
            if len(files) > 3:
                print(f"{sub_indent}... ({len(files) - 3} more files)")

# Run the main function
main()

## How to Use This Code

1. **Update API Keys**:
   - Edit the cell below
   - Replace `YOUR_ELEXON_API_KEY` with your actual Elexon API key
   - Replace `YOUR_NESO_API_KEY` with your actual NESO API key

2. **Run the Cell**:
   - After updating the API keys, just run the cell
   - The code will automatically:
     - Create an "elexon_neso_downloads" folder
     - Download data for the last 8 days
     - Save all files in organized folders
     - Print progress as it downloads
     - Show a summary when complete

3. **Check Results**:
   - All data will be saved in the "elexon_neso_downloads" folder
   - Files are organized by source (elexon/neso), dataset, and date
   - A summary file will be saved in the main folder

You don't need to run any other cells - this is a complete, self-contained solution.

# Quick Start Guide

Good news! I've created a virtual environment for you called `zmq_env` with PyZMQ installed successfully.

## To run this notebook:

1. **In VS Code, select the correct kernel:**
   - Look at the top-right corner of the notebook
   - Click on the current kernel or "Select Kernel" if none is selected
   - Choose "Python Environments..."
   - Select "zmq_env" from the list

2. **Run cells:**
   - Click on each cell and then click the ▶️ play button on the left side
   - Or press Shift+Enter with a cell selected

The PyZMQ package has been successfully installed (version 27.0.2).

Let's explore how ZeroMQ can help with your Elexon/NESO data processing needs!

# ZeroMQ for Elexon/NESO Data Analysis - Setup Instructions

## Before Running This Notebook

Based on the terminal errors you encountered, you need to set up your Python environment properly before running this notebook. Follow these steps:

1. **Install Python (if not already installed)**
   - Download and install from [python.org](https://www.python.org/downloads/)
   - Make sure to check "Add Python to PATH" during installation

2. **Create a Python virtual environment**
   - Open Terminal 
   - Navigate to your project directory:
     ```
     cd "/Users/georgemajor/Jibber Jabber ChatGPT/8_august_jibber_jabber"
     ```
   - Create a virtual environment:
     ```
     python3 -m venv venv
     ```
   - Activate the virtual environment:
     ```
     source venv/bin/activate
     ```

3. **Install required packages**
   - With the virtual environment activated, install PyZMQ:
     ```
     pip install pyzmq
     ```

4. **Select the correct kernel**
   - In VS Code, click on the kernel selector in the top-right corner
   - Select the Python kernel from your virtual environment ("venv")

After completing these steps, you should be able to run the notebook cells properly.

## Run Each Cell by Clicking the Play Button

To run a cell in this notebook:
1. Click on the cell you want to run
2. Click the ▶️ (play) button that appears to the left of the cell
3. Or press Shift+Enter while the cell is selected

Let's get started!

In [ ]:
import zmq
import time
import random
import threading
import json
import os
import sys
from datetime import datetime

# Display ZMQ version
print(f"ZeroMQ Version: {zmq.zmq_version()}")
print(f"PyZMQ Version: {zmq.__version__}")

ZeroMQ Version: 4.3.5
PyZMQ Version: 27.0.2


In [ ]:
# ZeroMQ (ZMQ) Messaging Library for Elexon/NESO Data Analysis

This notebook demonstrates how to use the ZeroMQ (ZMQ) messaging library in Python for efficient data processing and analysis of Elexon and NESO data.

## What is ZeroMQ?

ZeroMQ is a high-performance asynchronous messaging library, designed to be used in distributed or concurrent applications. It provides a message queue, but unlike message-oriented middleware, a ZeroMQ system can run without a dedicated message broker.

## Benefits for Elexon/NESO Data Processing:

1. **Asynchronous Data Collection**: Parallelize data retrieval from multiple APIs
2. **Efficient Distribution**: Use pub/sub pattern to distribute collected data to processing components
3. **Fault Tolerance**: Robust error handling for resilient data collection
4. **Scalability**: Scale out across distributed systems for large datasets

Let's explore how to implement these patterns for our data analysis workflow.

SyntaxError: unterminated string literal (detected at line 16) (1544514427.py, line 16)

## Section 1: Introduction to ZeroMQ Context

The ZeroMQ context is the container for all sockets in a process. It's thread-safe and manages the background I/O threads. Let's create a context and explore its configuration.

In [ ]:
# Create a ZeroMQ context
context = zmq.Context()

# The context is the container for all sockets in a process
# It's thread safe and can be used to create multiple sockets
print("Created ZeroMQ context")

# You can configure the context
context.set(zmq.IO_THREADS, 4)  # Set the number of I/O threads
context.set(zmq.MAX_SOCKETS, 1024)  # Set the maximum number of sockets

# Get current settings
io_threads = context.get(zmq.IO_THREADS)
max_sockets = context.get(zmq.MAX_SOCKETS)

print(f"Context settings: IO_THREADS={io_threads}, MAX_SOCKETS={max_sockets}")

# Important: Always terminate the context when done
# context.term()  # Commented out as we'll use it throughout the notebook

Created ZeroMQ context
Context settings: IO_THREADS=4, MAX_SOCKETS=1024


## Section 2: Creating Socket Connections

ZeroMQ supports various socket types for different messaging patterns. For Elexon/NESO data processing, we'll primarily use:

1. **REQ/REP**: For API requests and responses
2. **PUB/SUB**: For distributing data to multiple processing nodes
3. **PUSH/PULL**: For workload distribution in data processing pipeline

Let's explore the socket types and basic connection methods.

In [ ]:
# ZeroMQ supports various socket types for different messaging patterns:
socket_types = {
    "REQ/REP": "Request-Reply pattern for client-server communication",
    "PUB/SUB": "Publish-Subscribe pattern for one-to-many distribution",
    "PUSH/PULL": "Pipeline pattern for distributing work to workers",
    "DEALER/ROUTER": "Advanced asynchronous request-reply pattern",
    "PAIR": "Exclusive connection between two sockets"
}

for socket_type, description in socket_types.items():
    print(f"{socket_type}: {description}")

# Example: Creating a REQ socket (client)
req_socket = context.socket(zmq.REQ)
print("\nCreated REQ socket (client)")

# Example: Creating a REP socket (server)
rep_socket = context.socket(zmq.REP)
print("Created REP socket (server)")

# Socket options example
req_socket.set(zmq.LINGER, 0)  # Don't wait for unsent messages when closing
req_socket.set(zmq.RCVTIMEO, 1000)  # Receive timeout in milliseconds
req_socket.set(zmq.SNDTIMEO, 1000)  # Send timeout in milliseconds

# Connect and bind
# Note: In a real application, you would use actual addresses
print("\nConnection methods:")
print("- bind(): Used by the server to wait for connections")
print("- connect(): Used by the client to connect to a server")

# Example connection (just for demonstration)
try:
    rep_socket.bind("tcp://*:5555")
    print("Bound REP socket to tcp://*:5555")
    
    req_socket.connect("tcp://localhost:5555")
    print("Connected REQ socket to tcp://localhost:5555")
except zmq.ZMQError as e:
    print(f"ZMQ Error: {e}")
finally:
    # Clean up sockets when done
    req_socket.close()
    rep_socket.close()
    print("\nClosed sockets")

## Section 3: Message Patterns in ZeroMQ - Request-Reply

The Request-Reply pattern is ideal for API interactions with Elexon and NESO services. Let's implement a simple client-server example to demonstrate how this works.

In this example:
1. The server will simulate an API endpoint (like Elexon/NESO)
2. The client will send requests (like our data collector)
3. Both will run in separate threads to demonstrate asynchronous operation

In [ ]:
# Let's implement a simple request-reply pattern
print("\n=== Request-Reply Pattern Example ===")

def server_thread():
    """Server function to respond to client requests (simulates Elexon/NESO API)"""
    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.bind("tcp://*:5555")
    
    print("Server started, waiting for requests...")
    
    # Process 5 requests then exit
    for i in range(5):
        # Wait for client request
        message = socket.recv_string()
        print(f"Server received: {message}")
        
        # Simulate work (like API processing)
        time.sleep(0.5)
        
        # Send reply (like API response)
        socket.send_string(f"Response to '{message}'")
    
    socket.close()
    context.term()
    print("Server terminated")

def client_thread():
    """Client function to send requests and receive replies (simulates our data collector)"""
    # Give the server time to start
    time.sleep(1)
    
    context = zmq.Context()
    socket = context.socket(zmq.REQ)
    socket.connect("tcp://localhost:5555")
    
    # Send 5 requests
    for i in range(5):
        request = f"Data request #{i+1}"
        print(f"Client sending: {request}")
        socket.send_string(request)
        
        # Get reply
        reply = socket.recv_string()
        print(f"Client received: {reply}")
        
        # Small delay between requests
        time.sleep(0.2)
    
    socket.close()
    context.term()
    print("Client terminated")

# Start server and client in separate threads
server = threading.Thread(target=server_thread)
client = threading.Thread(target=client_thread)

server.start()
client.start()

# Wait for both threads to finish
server.join()
client.join()

## Section 4: Publish-Subscribe Pattern

The Publish-Subscribe pattern is excellent for distributing collected data to multiple analysis components. This pattern is useful when you want to:

1. Send Elexon data to multiple analysis modules simultaneously 
2. Filter data by topic (e.g., "weather" or "balancing" data)
3. Have loose coupling between data producers and consumers

Let's implement a simple example with a publisher and multiple subscribers:

In [ ]:
print("\n=== Publish-Subscribe Pattern Example ===")

def publisher_thread():
    """Publisher function to broadcast messages (simulates data collection service)"""
    context = zmq.Context()
    socket = context.socket(zmq.PUB)
    socket.bind("tcp://*:5556")
    
    # Give subscribers time to connect
    time.sleep(1)
    
    # Publish 10 messages
    for i in range(10):
        # Topics are 'balancing' or 'weather' (simulating different data types)
        topic = "balancing" if i % 2 == 0 else "weather"
        message = f"Data update #{i+1}"
        
        # Format: topic message
        socket.send_string(f"{topic} {message}")
        print(f"Published: {topic} {message}")
        
        time.sleep(0.2)
    
    socket.close()
    context.term()
    print("Publisher terminated")

def subscriber_thread(topic):
    """Subscriber function to receive messages with specific topic (simulates analysis modules)"""
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    
    # Subscribe to specific topic
    socket.setsockopt_string(zmq.SUBSCRIBE, topic)
    print(f"Subscriber for '{topic}' started")
    
    # Receive 5 messages then exit
    count = 0
    start_time = time.time()
    while count < 5 and time.time() - start_time < 3:
        try:
            # Set timeout for recv
            socket.RCVTIMEO = 1000
            message = socket.recv_string()
            print(f"Subscriber '{topic}' received: {message}")
            count += 1
        except zmq.ZMQError as e:
            if e.errno == zmq.EAGAIN:
                continue  # Timeout, try again
            else:
                print(f"Subscriber '{topic}' error: {e}")
                break
    
    socket.close()
    context.term()
    print(f"Subscriber '{topic}' terminated")

# Start publisher and subscribers in separate threads
publisher = threading.Thread(target=publisher_thread)
balancing_subscriber = threading.Thread(target=subscriber_thread, args=("balancing",))
weather_subscriber = threading.Thread(target=subscriber_thread, args=("weather",))

publisher.start()
balancing_subscriber.start()
weather_subscriber.start()

# Wait for all threads to finish
publisher.join()
balancing_subscriber.join()
weather_subscriber.join()

## Application to Elexon/NESO Data Collection

Let's now examine how we can apply these ZeroMQ patterns to create an efficient data collection and processing system for Elexon and NESO data.

### Data Collection Architecture

1. **Data Collection Service**:
   - Uses REQ/REP pattern to interact with Elexon/NESO APIs
   - Implements robust error handling and retry logic
   - Collects data from multiple endpoints in parallel

2. **Data Distribution Service**:
   - Uses PUB/SUB pattern to distribute collected data
   - Separates data by type (balancing, weather, system warnings, etc.)
   - Allows multiple analysis components to subscribe to relevant data

3. **Data Processing Pipeline**:
   - Uses PUSH/PULL pattern for distributed processing
   - Scales horizontally for large data processing tasks
   - Handles data validation, transformation, and loading to BigQuery

Let's implement a simplified example of this architecture:

In [ ]:
print("\n=== Elexon/NESO Data Collection System Example ===")

# Simulated data for our example
sample_data = {
    "balancing": [
        {"timestamp": "2025-08-24T10:00:00", "value": 245.67, "unit": "MW"},
        {"timestamp": "2025-08-24T10:30:00", "value": 250.12, "unit": "MW"},
        {"timestamp": "2025-08-24T11:00:00", "value": 242.89, "unit": "MW"}
    ],
    "weather": [
        {"timestamp": "2025-08-24T10:00:00", "location": "London", "temp": 22.5, "wind_speed": 5.2},
        {"timestamp": "2025-08-24T10:30:00", "location": "London", "temp": 23.1, "wind_speed": 4.8},
        {"timestamp": "2025-08-24T11:00:00", "location": "London", "temp": 23.8, "wind_speed": 4.5}
    ]
}

# 1. Data Collection Service (API client)
def api_client():
    """Simulates making API calls to Elexon/NESO services"""
    context = zmq.Context()
    socket = context.socket(zmq.PUSH)
    socket.bind("tcp://*:5557")  # Push collected data to the distribution service
    
    print("API Client started - collecting data...")
    
    # Simulate collecting different types of data
    for data_type, data_list in sample_data.items():
        for data_item in data_list:
            # Create a message with data type and payload
            message = {
                "type": data_type,
                "data": data_item,
                "collected_at": datetime.now().isoformat()
            }
            
            # Send the data to the distribution service
            socket.send_json(message)
            print(f"Collected {data_type} data: {data_item}")
            time.sleep(0.5)  # Simulate API rate limiting
    
    # Send termination message
    socket.send_json({"type": "terminate"})
    
    socket.close()
    context.term()
    print("API Client terminated")

# 2. Data Distribution Service
def distribution_service():
    """Receives data from collectors and distributes to processors by type"""
    context = zmq.Context()
    
    # Socket to receive data from collectors
    receiver = context.socket(zmq.PULL)
    receiver.connect("tcp://localhost:5557")
    
    # Socket to publish data to processors
    publisher = context.socket(zmq.PUB)
    publisher.bind("tcp://*:5558")
    
    print("Distribution Service started...")
    
    # Process messages until termination signal
    while True:
        # Receive collected data
        message = receiver.recv_json()
        
        # Check for termination message
        if message.get("type") == "terminate":
            # Forward termination to processors
            publisher.send_json({"type": "terminate"})
            break
        
        # Get data type (balancing, weather, etc.)
        data_type = message.get("type")
        
        # Publish data with type as topic
        print(f"Distributing {data_type} data...")
        publisher.send_json(message)
    
    receiver.close()
    publisher.close()
    context.term()
    print("Distribution Service terminated")

# 3. Data Processors (one for each data type)
def data_processor(data_type):
    """Processes data of a specific type"""
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    socket.connect("tcp://localhost:5558")
    
    # Subscribe to messages with our data type
    socket.setsockopt_string(zmq.SUBSCRIBE, "")  # In a real scenario, we would filter by topic
    
    print(f"{data_type.capitalize()} Processor started...")
    
    # Process data until termination signal
    while True:
        try:
            # Receive published data
            message = socket.recv_json()
            
            # Check for termination message
            if message.get("type") == "terminate":
                break
            
            # Only process our data type
            if message.get("type") == data_type:
                data = message.get("data", {})
                # Simulate processing
                print(f"{data_type.capitalize()} Processor: Processing data from {data.get('timestamp', 'unknown')}")
                # In a real system, we would transform the data and load it to BigQuery
        except zmq.ZMQError as e:
            print(f"{data_type.capitalize()} Processor error: {e}")
            break
    
    socket.close()
    context.term()
    print(f"{data_type.capitalize()} Processor terminated")

# Start all components in separate threads
api_thread = threading.Thread(target=api_client)
distribution_thread = threading.Thread(target=distribution_service)
balancing_processor_thread = threading.Thread(target=data_processor, args=("balancing",))
weather_processor_thread = threading.Thread(target=data_processor, args=("weather",))

# Start threads
api_thread.start()
distribution_thread.start()
balancing_processor_thread.start()
weather_processor_thread.start()

# Wait for all threads to finish
api_thread.join()
distribution_thread.join()
balancing_processor_thread.join()
weather_processor_thread.join()

print("Elexon/NESO Data Collection System Example completed")

## Conclusion and Next Steps

In this notebook, we've demonstrated how ZeroMQ can be used to create a distributed, scalable system for collecting and processing Elexon and NESO data. The key benefits of this approach include:

1. **Improved Performance**: Asynchronous, non-blocking I/O for efficient data collection
2. **Scalability**: Easily distribute workloads across multiple processes or machines
3. **Resilience**: Built-in error handling and timeout mechanisms
4. **Flexibility**: Various messaging patterns to suit different parts of the system

### Next Steps for Implementation

1. **Integrate with Real APIs**: Replace simulated data with actual Elexon/NESO API calls
2. **Add Authentication**: Implement secure communication using ZeroMQ's CURVE encryption
3. **Error Recovery**: Add persistent queues to handle service interruptions
4. **Monitoring**: Implement a monitoring system to track data collection progress
5. **Deployment**: Package components as containerized services for easy deployment

With these improvements, you'll have a robust, scalable system for collecting and analyzing energy market data.